## LDA 01

참고: 이기창, 2019, 한국어 임베딩

In [ ]:
from gensim import corpora
from preprocess import get_tokenizer     

In [ ]:
# pip install preprocessing         # 설치 오류

In [ ]:
# pip install </home/aiffel-dj23/preprocess-2.0.0>

In [ ]:
# 모델피처 생성

corpus_frame = ''

documents, tokenized_corpus = [], []
tokenizer = get_tokenizer("mecab")

with open(courpus_frame, 'r', encoding='utf-8') as f:
    for document in f:
        tokens = list(set(tokenizer.morphs(document.strip())))
        documents.append(document)
        tokenized_corpus.append(token)
dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]


In [ ]:
# LDA 학습 및 결과 확인

from gensim.models import ldamulticore

LDA =ldamulticore.LdaMulticore(corpus, id2word=dictionary), num_topic =30, workers=4)

all_topics = LDA.get_document_topics(corpus, minimum_probability=0.5, 
                                     per_word_topics= False)

for doc_idx, topic in enumerate(all_topics[:5]):
    print(doc_idx, topic)


In [ ]:
# LDA 평가 모듈 선언

from models.sent_eval import LDAEvaluator
model = LDAEvaluator("data/sentence-embeddings/lda/lda")

In [ ]:
# 토픽별 문서 확인

model.show_topic_docs(topic_id=0)

In [ ]:
#토픽별 단어 확인

model.show_topic_words(topic_id=0)

In [ ]:
# 새로운 문서의 토픽 확인

model.show_new_document_topic(["너무 사랑스러운 영화", "인생을 말하는 영화"])

## LDA 02

참고: https://joyhong.tistory.com/138

In [ ]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
import os
from os.path import join
import pandas as pd

In [ ]:
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

In [ ]:
pip install pyLDAvis==3.2.2

In [ ]:
# 시각화를 위해 설치
import pyLDAvis.gensim

In [ ]:
# 명사만 추출한 csv 데이터 준비

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = [] 
    model_list = []
    # ditionary는 Gensim dictionary, corpus는 Gensim corpus
    # text는 list of input texts, limit 는 max of topics
    
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


def find_optimal_number_of_topics(dictionary, corpus, processed_data):
    limit = 40;
    start = 2;
    step = 6;
    
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step)

    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

    
if __name__ == '__main__':
    # 명사만으로 구성된 파일 open 
    processed_data = [sent.strip().split(",") for sent in tqdm(open('./Desktop/project/data/test01.csv', 'r', encoding='utf-8').readlines())]

    # 정수 인코딩과 빈도수 생성
    dictionary = corpora.Dictionary(processed_data)

    # 출현빈도가 적거나 자주 등장하는 단어는 제거
    dictionary.filter_extremes(no_below=10, no_above=0.05)
    corpus = [dictionary.doc2bow(text) for text in processed_data]
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

     # 최적의 토픽 수 찾기
    find_optimal_number_of_topics(dictionary, corpus, processed_data)
    exit()

In [ ]:
if __name__ == '__main__':
    processed_data = [sent.strip().split(",") for sent in tqdm(open('./tokenized_data.csv', 'r', encoding='utf-8').readlines())]
    # 정수 인코딩과 빈도수 생성
    dictionary = corpora.Dictionary(processed_data)

    # 출현빈도가 적거나 자주 등장하는 단어 제거
    dictionary.filter_extremes(no_below=10, no_above=0.05)
    corpus = [dictionary.doc2bow(text) for text in processed_data]
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # 최적의 토픽 수 찾기 => 14개로 나옴

    # 최적의 토픽 수 찾기 = 14개 나옴
    
    perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
    coherence_logger = CoherenceMetric(corpus=corpus, coherence="u_mass", logger='shell')

    lda_model = LdaModel(corpus, id2word=dictionary, num_topics=14, passes=30, callbacks=[coherence_logger, perplexity_logger])

    topics = lda_model.print_topics(num_words=5)

    for topic in topics:
        print(topic)
    
    # Compute Coherence Score using c_v
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score (c_v): ', coherence_lda)

    # Compute Coherence Score using UMass
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=dictionary, coherence="u_mass")
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score (u_mass): ', coherence_lda)
   
    # 저장  
    pickle.dump(corpus, open('./data/lda/lda_corpus.pkl', 'wb'))
    dictionary.save('./data/lda/lda_dictionary.gensim')
    lda_model.save('./data/lda/lda_model.gensim')

    # pyLDAvis html 저장
    lda_visualization = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(lda_visualization, './data/lda/독립유공자공적조서_lda.html')


In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.display(lda_visualization)

### 결과
u_mass는 0에 가까울수록 완벽한 일관성을 가진다는 의미이고 c_v는 0과 1사이로 0.55 정도면 준수하다.